In [38]:
%load gundong_class_tensor.py
'''
    进行动态预测的代码，用张量优化的版本
'''
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.dates as mdate
from numpy import *
from matplotlib.pylab import rcParams
plt.rcParams['axes.unicode_minus']=False
rcParams['font.sans-serif'] = 'kaiti'

from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.stats.diagnostic
from statsmodels.tsa.api import VAR


class gundong_tensor():
    def __init__(self, data, gundong_time, k_lag):
        self.row = data.shape[0] # 行长度
        self.column = data.shape[1] # 列长度
        self.data = data
        self.gundong_time = gundong_time # 滚动选择的时间
        self.k_lag = k_lag
        self.save_data_coef = np.zeros((self.row-self.gundong_time+1, self.column, self.k_lag*self.column))
        self.save_data_cov = np.zeros((self.row-self.gundong_time+1, self.column, self.column))
        self.save_data_result = np.zeros((self.row-self.gundong_time+1, self.column, self.column))

    def VAR(self):
        '''
        实现滚动计算 k-lag 的 VAR 模型
        并且保存矩阵的系数以及相关系数矩阵
        实现了 k-lag>1 时的向量值回归模型

        '''
        for i in range(self.gundong_time, self.row+1,1):
            datai = self.data.iloc[i-self.gundong_time:i,:]
            model = VAR(datai, dates=None)
            # 滞后 k_lag 个单位计算
            results = model.fit(self.k_lag)
            coef = results.params
            self.save_data_coef[i-self.gundong_time,:,:]= coef.iloc[1:1+self.k_lag*self.column,:].T
            self.save_data_cov[i-self.gundong_time,:,:] = results.sigma_u


    def calculate_multiply(self):
        # 初始的 A_0,...A_{1-p}

        # 第一个分块矩阵是单位阵
        matrix_identity = np.zeros((self.k_lag*self.column, self.column))
        matrix_identity[0:self.column,:] = np.identity(self.column)
        matrix_identity = np.expand_dims(matrix_identity,0).repeat(self.row-self.gundong_time+1, axis=0)
        matrix_left = np.matmul(matrix_identity, self.save_data_coef)
        matrix_right = np.zeros((self.k_lag*self.column, self.k_lag*self.column))
        for j in range(1,self.k_lag):
            matrix_right[j*self.column:(j+1)*self.column, (j-1)*self.column:j*self.column] = np.identity(self.column)
        matrix_right = np.expand_dims(matrix_right,0).repeat(self.row-self.gundong_time+1, axis=0)
        matrix_multiply = matrix_left+matrix_right
        return matrix_multiply


    def cal_overflow(self, predict_time):
        '''
            适用于不同 k_lag 的向量自回归模型
            
            张量乘法运算 a*b*c 维张量 matmul a*c*d 维张量结果是 a*b*d 维张量

            Args:
                predict_time: 预测天数
        '''
        self.predict_time = predict_time
        # 初始的 A_h 矩阵 
        self.A_h = np.zeros((self.row-self.gundong_time+1, self.k_lag*self.column, self.column))
        self.A_h[:, 0:self.column,:] = np.identity(self.column)
        # 得到
        matrix_multiply = self.calculate_multiply()
        temp = np.matmul(self.A_h[:,0:self.column,:],self.save_data_cov)
        sum_top = temp*temp
        # 得到一个对角阵
        sigma_jj = self.save_data_cov.diagonal(axis1=1, axis2=2)
        sigma_jj = np.apply_along_axis(np.diag, 1, sigma_jj)
        # A_h * cov * A_h'
        temp_bottom = np.matmul(temp, self.A_h[:,0:self.column,:].transpose(0,2,1))
        # 每行元素都是对角线元素
        temp_bottom = temp_bottom.diagonal(axis1=1, axis2=2)[:,np.newaxis].transpose(0,2,1).repeat(self.column,2)
        # * sigma_jj
        sum_bottom = np.matmul(temp_bottom, sigma_jj)
        for h in range(self.predict_time-1):
            self.A_h = np.matmul(matrix_multiply, self.A_h)
            temp = np.matmul(self.A_h[:,0:self.column,:], self.save_data_cov)
            sum_top = sum_top + temp*temp
            temp_bottom = np.matmul(temp, self.A_h[:,0:self.column,:].transpose(0,2,1))
            temp_bottom = temp_bottom.diagonal(axis1=1, axis2=2)[:,np.newaxis].transpose(0,2,1).repeat(self.column,2)
            sum_bottom = sum_bottom + np.matmul(temp_bottom, sigma_jj)

        self.save_data_result = sum_top/sum_bottom

        def standard_overflow(self):
            '''
            计算溢出指数的比重


            '''
            pass


    def save_data(self, path):
        np.save(path+'save_data_coef',self.save_data_coef)
        np.save(path+'save_data_cov',self.save_data_cov)
        np.save(path+'save_data_result',self.save_data_result)



## 标准处理

In [24]:
file_path='./now/zong.csv'
data = pd.read_csv(file_path, index_col = 0)

In [60]:
data

,CL,DC,DX,GY,GYSY,JR,KX,NY,RC,XX,YL
index,,,,,,,,,,,
2014/1/2,8.800000e-10,8.330000e-10,1.050000e-10,6.480000e-10,6.760000e-10,9.100000e-11,5.750000e-10,1.800000e-10,4.480000e-10,5.350000e-10,4.790000e-10
2014/1/3,9.480000e-10,1.050000e-09,1.260000e-10,7.010000e-10,7.290000e-10,1.090000e-10,6.260000e-10,2.050000e-10,4.640000e-10,4.640000e-10,4.200000e-10
2014/1/6,1.340000e-09,1.360000e-09,1.420000e-10,1.010000e-09,9.390000e-10,1.140000e-10,8.760000e-10,2.800000e-10,7.270000e-10,6.860000e-10,5.840000e-10
2014/1/7,1.240000e-09,1.240000e-09,1.060000e-10,9.890000e-10,9.500000e-10,1.230000e-10,8.320000e-10,2.460000e-10,6.620000e-10,7.240000e-10,5.680000e-10
2014/1/8,1.130000e-09,1.160000e-09,1.260000e-10,8.620000e-10,8.450000e-10,1.110000e-10,7.590000e-10,2.900000e-10,5.930000e-10,5.380000e-10,4.530000e-10
...,...,...,...,...,...,...,...,...,...,...,...
2019/6/24,3.870000e-10,3.140000e-10,3.890000e-11,3.240000e-10,3.100000e-10,2.950000e-11,3.360000e-10,1.300000e-10,1.300000e-10,2.020000e-10,2.690000e-10
2019/6/25,4.280000e-10,4.430000e-10,5.590000e-11,3.950000e-10,4.080000e-10,3.980000e-11,4.260000e-10,1.430000e-10,1.590000e-10,2.580000e-10,3.470000e-10
2019/6/26,3.870000e-10,3.880000e-10,6.850000e-11,4.170000e-10,3.850000e-10,3.850000e-11,4.210000e-10,1.230000e-10,1.560000e-10,0.000000e+00,3.240000e-10


In [58]:
[column for column in data]

['CL', 'DC', 'DX', 'GY', 'GYSY', 'JR', 'KX', 'NY', 'RC', 'XX', 'YL']

In [39]:
gundong_data = gundong_tensor(data, gundong_time=360, k_lag=2)

In [40]:
gundong_data.VAR()

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

...

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_mode

In [41]:
gundong_data.cal_overflow(predict_time=10)

In [43]:
gundong_data.save_data_result[1].shape

(11, 11)

In [11]:
gundong_data.save_data_cov.shape

(979, 11, 11)

In [55]:
model = VAR(data)
lagorder=model.select_order(15)

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [56]:
lagorder.summary()

,AIC,BIC,FPE,HQIC
0,-502.7,-502.6,4.830e-219,-502.7
1,-506.3,-505.8*,1.258e-220,-506.1
2,-506.6,-505.6,9.495e-221,-506.2
3,-506.8,-505.3,8.171e-221,-506.2
4,-507.1,-505.1,6.030e-221,-506.3*
5,-507.0,-504.6,6.428e-221,-506.1
6,-507.0,-504.1,6.817e-221,-505.9
7,-507.0,-503.7,6.334e-221,-505.8
8,-507.1,-503.3,5.763e-221,-505.7
9,-507.1,-502.7,6.124e-221,-505.4


In [59]:
result = model.fit(data)
result.test_causality('CL', [column for column in data], kind='f')

TypeError: 'DataFrame' object cannot be interpreted as an integer

In [53]:
results = model.fit(maxlags=15, ic='aic')
results.params

,CL,DC,DX,GY,GYSY,JR,KX,NY,RC,XX,YL
const,2.500248e-11,3.515510e-11,1.170061e-11,1.608622e-11,3.890539e-11,4.677898e-12,4.587391e-11,1.311624e-11,2.216507e-11,2.766588e-11,1.115165e-10
L1.CL,3.251197e-01,1.428583e-02,4.774407e-03,-3.039946e-02,1.090687e-01,5.470627e-03,9.345505e-02,1.643842e-02,1.105427e-02,2.119966e-02,5.739955e-03
L1.DC,3.783625e-02,4.595860e-01,8.364326e-04,3.517927e-02,1.018455e-02,-1.664965e-03,6.758227e-02,-1.073833e-02,-9.033358e-03,-8.480050e-02,5.334413e-02
L1.DX,1.225636e-01,7.400252e-02,4.065217e-01,1.083143e-01,-1.355462e-02,-2.407198e-02,1.389245e-01,-5.460040e-02,5.558404e-02,2.441135e-02,-1.230089e-01
L1.GY,1.235103e-02,8.430538e-02,-5.103817e-04,1.431582e-01,3.808526e-02,1.885632e-03,2.730411e-02,-1.911952e-02,-8.234018e-03,2.419343e-02,-8.067012e-02
...,...,...,...,...,...,...,...,...,...,...,...
L15.KX,1.813056e-02,7.669092e-02,2.146201e-02,1.105841e-01,6.221309e-02,-2.011870e-04,7.058327e-02,1.989952e-02,2.852132e-02,2.680961e-02,-1.026764e-02
L15.NY,-6.671538e-02,1.823415e-02,7.946990e-03,-1.922442e-01,-1.617260e-01,6.333126e-04,-2.288319e-01,-2.762056e-02,-1.661942e-01,-1.288795e-01,-4.130734e-01
L15.RC,2.242394e-01,1.106878e-01,-1.164123e-02,8.915259e-02,8.710085e-02,-6.432484e-04,1.934434e-01,-3.591225e-03,1.538687e-01,1.744751e-01,2.590862e-01
L15.XX,1.603627e-02,2.393965e-02,6.971465e-04,-1.815946e-02,-1.126981e-02,-3.161476e-04,-3.703468e-02,-4.531084e-03,3.381331e-02,-1.398088e-03,1.178809e-01


In [ ]:
l

In [14]:
k_lag = 2
column = 11
row = 1338
gundong_time = 360
predict_time = 10

# def calculate_multiply():
    # 初始的 A_0,...A_{1-p}

    # 第一个分块矩阵是单位阵
matrix_identity = np.zeros((k_lag*column, column))
matrix_identity[0:column,:] = np.identity(column)
matrix_identity = np.expand_dims(matrix_identity,0).repeat(row-gundong_time+1, axis=0)
matrix_left = np.matmul(matrix_identity, gundong_data.save_data_coef)
matrix_right = np.zeros((k_lag*column, k_lag*column))
for j in range(1,k_lag):
    matrix_right[j*column:(j+1)*column, (j-1)*column:j*column] = np.identity(column)
matrix_right = np.expand_dims(matrix_right,0).repeat(row-gundong_time+1, axis=0)
matrix_multiply = matrix_left+matrix_right
    # return matrix_multiply

matrix_multiply = calculate_multiply()
matrix_multiply.shape

(979, 22, 22)

In [21]:



# 初始的 A_h 矩阵 
A_h = np.zeros((row-gundong_time+1, k_lag*column, column))
A_h[:,0:column,:] = np.identity(column)
# 得到

temp = np.matmul(A_h[:,0:column,:],gundong_data.save_data_cov)
sum_top = temp*temp
# 得到一个对角阵
sigma_jj = gundong_data.save_data_cov.diagonal(axis1=1, axis2=2)
sigma_jj = np.apply_along_axis(np.diag, 1, sigma_jj)
# A_h * cov * A_h'
temp_bottom = np.matmul(temp, A_h[:,0:column,:].transpose(0,2,1))
# 每行元素都是对角线元素
# ?
temp_bottom = temp_bottom.diagonal(axis1=1, axis2=2)[:,np.newaxis].transpose(0,2,1).repeat(column,2)
# * sigma_jj
sum_bottom = np.matmul(temp_bottom, sigma_jj)
for h in range(predict_time-1):
    A_h = np.matmul(matrix_multiply, A_h)
    temp = np.matmul(A_h[:,0:column,:], gundong_data.save_data_cov)
    sum_top = sum_top + temp*temp
    temp_bottom = np.matmul(temp, A_h[:,0:column,:].transpose(0,2,1))
    temp_bottom = temp_bottom.diagonal(axis1=1, axis2=2)[:,np.newaxis].transpose(0,2,1).repeat(column,2)
    sum_bottom = sum_bottom + np.matmul(temp_bottom, sigma_jj)

save_data_result = sum_top/sum_bottom

In [17]:
temp = np.arange(12).reshape(3,2,2)
temp

array([[[ 0,  1],
        [ 2,  3]],

       [[ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11]]])

In [18]:
temp.diagonal(axis1=1,axis2=2)[:,np.newaxis].transpose(0,2,1).repeat(5,2)

array([[[ 0,  0,  0,  0,  0],
        [ 3,  3,  3,  3,  3]],

       [[ 4,  4,  4,  4,  4],
        [ 7,  7,  7,  7,  7]],

       [[ 8,  8,  8,  8,  8],
        [11, 11, 11, 11, 11]]])

In [61]:
import numpy as np
import pandas
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
mdata = sm.datasets.macrodata.load_pandas().data
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
from statsmodels.tsa.base.datetools import dates_from_str
quarterly = dates_from_str(quarterly)
mdata = mdata[['realgdp','realcons','realinv']]
mdata.index = pandas.DatetimeIndex(quarterly)
data = np.log(mdata).diff().dropna()

###### data